## **Introducción** 

This notebook runs basic funcionalities of the code.

Use a GPU envoronment for better perfomance. 

For more information contact me at mxbastidas@unal.edu.co

## **Github connection**

accessing to ssh key for bird_zero_shot_tracking.git

In [2]:
! mkdir -p /root/.ssh

upload id_rsa the file to the colab files at the rigth menu on this notebook

In [3]:
!mv id_rsa.pub /root/.ssh/

In [4]:
! ssh-keyscan -t rsa github.com >> ~/.ssh/known_hosts

# github.com:22 SSH-2.0-babeld-8f627ebe


In [6]:
! chmod go-rwx /root/.ssh/id_rsa.pub

Cheking if the connection was succesful

In [7]:
!ssh -T git@github.com

git@github.com: Permission denied (publickey).


Configuring the github user data

In [9]:
!git config --global user.email "mxbastidasr@unal.edu.co"
!git config --global user.name "mxbastidasr"

Cloning repo

In [10]:
!git clone git@github.com:mxbastidasr/bird_zero_shot_tracking.git

Cloning into 'bird_zero_shot_tracking'...
git@github.com: Permission denied (publickey).
fatal: Could not read from remote repository.

Please make sure you have the correct access rights
and the repository exists.


## **Instaling dependencies**

In [26]:
!pip install git+https://github.com/openai/CLIP.git
!pip install ultralytics
!pip install transformers
!pip install ipympl

## **Bird zero shot tracking**

In [ ]:
%cd bird_zero_shot_tracking/

/content/bird_zero_shot_tracking


In [29]:
import os 
import torch
import pandas as pd

from clip_object_tracker import DetectionAndTracking

In [ ]:
class Args:
  weights = "models/yolov8x.pt"# help='model.pt path(s)'
  names = 'coco.names' # help='yolov4 names file, file path'
  source ='data/images' # help='source'
  img_size = 640 # help='inference size (pixels)'
  confidence=0.40 # help='object confidence threshold'
  overlap=0.30 # help='IOU threshold for NMS'
  thickness=3 # help='Thickness of the bounding box strokes')
  device='' # help='cuda device, i.e. 0 or 0,1,2,3 or cpu')
  view_img = False # help='display results')
  save_txt = False # help='save results to *.txt')
  save_conf = False #help='save confidences in --save-txt labels')
  classes = None # help='filter by class: --class 0, or --class 0 2 3')
  agnostic_nms = False # help='class-agnostic NMS')
  augment = False #help='augmented inference')
  update = False # help='update all models')
  project = 'runs/detect' # help='save results to project/name')
  name = 'exp' # help='save results to project/name')
  exist_ok = False # help='existing project/name ok, do not increment')
  nms_max_overlap = 1.0 # help='Non-maxima suppression threshold: Maximum detection overlap.')
  max_cosine_distance = 0.4 # help='Gating threshold for cosine distance metric (object appearance).')
  nn_budget = None # help='Maximum size of the appearance descriptors allery. If None, no budget is enforced.')
  info = False # help='Print debugging info.')
  detection_engine = "yolov8" # help="Which engine you want to use for object detection (yolov7, yolov5, yolov4, roboflow).")
  clip_labels = ["hummingbird", "flower"]
opt = Args()

In [28]:
opt.weights = "models/yolov8x.pt"
opt.detection_engine = "yolov8"
opt.save_txt = True
opt.confidence = 0.7
opt.overlap = 0.1
opt.clip_labels = ["hummingbird", "flower","glass","plastic", "not a bird"]
video_detection = DetectionAndTracking(opt)

openai/clip-vit-large-patch14


`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.


100%|██████████| 131M/131M [00:00<00:00, 274MB/s]
YOLOv8x summary (fused): 268 layers, 68200608 parameters, 0 gradients, 257.8 GFLOPs


In [31]:
project_path = "runs/detect"

In [36]:
full_path = "data/video/hummingbird_hq.mp4"
file_name = "hummingbird_hq.mp4"
with torch.no_grad():  
  video_detection(full_path,project_path, file_name, save_img=True)

KeyboardInterrupt: ignored

In [ ]:
df = pd.read_table(os.path.join(project_path,f"{file_name}/labels/full_labels.txt"), sep = ';', names=('frame', 'track', 'class','bbox','space'))

In [ ]:
from IPython.display import HTML
from base64 import b64encode
mp4 = open(os.path.join(project_path,f"{file_name}/{file_name}.mp4",'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)